# Geometrias das faixas de pedestre

In [1]:
import geopandas as gpd
import rasterio
import numpy as np
from sklearn.cluster import DBSCAN
from shapely.geometry import MultiPoint, LineString, Point
from shapely.ops import substring, split, nearest_points
from scipy.spatial import distance
import math
import os
from rasterio.enums import Resampling
import glob
import re

/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
url = 'https://github.com/AndaSampa/poligono-de-vias/raw/master/resultado/90_poligono_de_vias_de_vila_mariana.gpkg'
gdf_pvias = gpd.read_file(url)
gdf_pvias_buffered = gdf_pvias.buffer(-1).unary_union
gdf_pvias_union = gpd.GeoSeries(gdf_pvias.unary_union)

In [3]:
url_rotas_mediais = 'https://github.com/AndaSampa/rotas-em-calcadas/raw/main/resultados/rotas-mediais-vila-mariana.gpkg'
gdf_rotas_mediais = gpd.read_file(url_rotas_mediais)

In [4]:
rasters = glob.glob('resultados/raster/faixas-*.jp2')

In [5]:
faixas_list = []
epilson = 6

In [6]:
def quadri_metrics(coords):
    
    distances, angles = [], []

    for i, v in enumerate(coords):
        if i == 0 : continue
        distances.append(distance.euclidean(coords[i-1], v))
        angle_dif = np.array(coords[i-1]) - np.array(v)
        angles.append(math.atan2(angle_dif[0], angle_dif[1]))

    distances = np.array(distances)

    return np.min(distances), np.max(distances), angles[distances.argmin()], angles[distances.argmax()]

    

In [7]:
for f in rasters:
    dataset = rasterio.open(f)
    scm = re.findall(r'[0-9]+-[0-9]+', f)[0]
    print(f'Processando: SCM {scm}')
    data = dataset.read()
    faixas_ij = np.array([np.where(data == 1)[2], np.where(data == 1)[1]]).transpose()
    t = dataset.transform
    faixas_xy = faixas_ij * (t[0], t[4]) + (t[2], t[5]) + (t[0]/2, t[4]/2)
    faixas_list.append(faixas_xy)

    geometries = gpd.points_from_xy(x=faixas_xy[:, 0], y=faixas_xy[:, 1], crs='EPSG:31983')
    gdf_faixas = gpd.GeoDataFrame(geometry=geometries)
    
    gdf_faixas = gpd.sjoin(gdf_faixas, gdf_pvias, how='left', predicate='within')
    gdf_faixas.loc[gdf_faixas.index_right.isna(), ['index_right']] = 0
    gdf_faixas.loc[:, ['index_right']] = gdf_faixas.loc[:, ['index_right']] * (epilson + 1) + epilson + 1

    coords = np.array([gdf_faixas.geometry.x, gdf_faixas.geometry.y, gdf_faixas.index_right]).transpose()

    clusters = DBSCAN(eps=epilson).fit_predict(coords)
    
    gdf_faixas.loc[:, ['cluster']] = clusters

    # Remove ruido dos clusters
    gdf_faixas = gdf_faixas[gdf_faixas.cluster != -1]

    # gdf_faixas.to_file(f'resultados/vetor/faixas-{scm}.gpkg', driver='GPKG')

    faixas_poly = gdf_faixas.groupby('cluster')['geometry'].apply(lambda x: MultiPoint(list(x)).minimum_rotated_rectangle)
    
    # Remove polígonos com área menor que 2m
    faixas_poly = faixas_poly[faixas_poly.geometry.area > 2]

    # faixas_poly.to_file(f'resultados/vetor/faixas-{scm}.gpkg', driver='GPKG')

    # Remove os polígonos que o ponto de inacessibilidade esteja 
    # fora de um buffer negativo de 1m do polígono de via
    faixas_poly = faixas_poly[faixas_poly.geometry.apply(lambda x: x.representative_point()).intersects(gdf_pvias_buffered)]

    # faixas_poly[:, ['largura', 'extensão', 'angulo da largura', 'angulo da extensão']] = \
    #     quadri_metrics()

    faixas_poly = gpd.GeoDataFrame(faixas_poly.geometry.exterior.apply(lambda x: quadri_metrics(x.coords)).to_list(), 
                                    index=faixas_poly.index, 
                                    geometry=faixas_poly.geometry, 
                                    columns=['largura', 'extensão', 'angulo da largura', 'angulo da extensão'])

    # 1. Encontrar a rota medial mais próxima
    medial_proxima = faixas_poly.sjoin_nearest(gdf_rotas_mediais, how='left', max_distance=80.0, distance_col='distancia')
    # Removendo duplicados
    medial_proxima = medial_proxima[~medial_proxima.index.duplicated(keep='first')]
    # Removendo sem associação
    medial_proxima = medial_proxima[medial_proxima.index_right.notna()]

    # 2. Obter os pontos entre a rota medial e o polígono
    linha_conexao_1 = medial_proxima.apply(lambda x: LineString(nearest_points(gdf_rotas_mediais.iloc[int(x.index_right)].geometry,x.geometry)), axis=1)

    # 3. Obter o ponto invertido dentro do polígono
    faixas_poly.loc[:, ['pt_invertido_norm']] = faixas_poly.apply(lambda x: x.geometry.boundary.project(Point(linha_conexao_1[x.name].coords[0]), normalized=True), axis=1)
    faixas_poly.loc[:, ['pt_invertido_norm']] += 0.5
    
    faixas_poly.loc[faixas_poly.pt_invertido_norm > 1.0, 'pt_invertido_norm'] -= 1.0

    try:
        pt_2 = gpd.GeoDataFrame(geometry=faixas_poly.apply(lambda x: x.geometry.boundary.interpolate(x.pt_invertido_norm, normalized=True), axis=1))
    except:
        pt_2 = gpd.GeoDataFrame(geometry=[])
    
    # 4. Conectar o ponto invertido do polígono às rotas mediais unificadas
    medial_proxima = pt_2.sjoin_nearest(gdf_rotas_mediais, how='left', max_distance=80.0, distance_col='distancia')
    # Removendo duplicados
    medial_proxima = medial_proxima[~medial_proxima.index.duplicated(keep='first')]
    # Removendo sem associação
    medial_proxima = medial_proxima[medial_proxima.index_right.notna()]

    linha_conexao_2 = medial_proxima.apply(lambda x: LineString(nearest_points(x.geometry,
                                                                                gdf_rotas_mediais.iloc[x.index_right].geometry,)), axis=1)
                                                                                
    # 5. Desenhar a linha entre todos os pontos
    pt_1 = linha_conexao_1.apply(lambda x: x.coords[0]).to_numpy()
    pt_2 = linha_conexao_2.apply(lambda x: x.coords[1]).to_numpy()

    try:
        linhas_de_travessia = gpd.GeoDataFrame(geometry=[LineString([x[0], x[1]]) for x in np.stack([pt_1, pt_2], axis=-1)])
        linhas_de_travessia.to_file(f'resultados/vetor/linhas-de-travessia-{scm}.gpkg', driver='GPKG')

        # 6. Gerar um offset com a largura da faixa a partir da linha de travessia
        travessia_buffered = gpd.GeoDataFrame(geometry=linhas_de_travessia.apply(lambda x: 
                                x.geometry.buffer(faixas_poly.reset_index().largura[x.name]/2, cap_style=3), axis=1))
        
        # 7. Cortar o novo polígono para ficar contido no polígono de vias                                    
        gpd.clip(travessia_buffered, gdf_pvias_union).to_file(f'resultados/vetor/faixas-poly-{scm}.gpkg', driver='GPKG')

    except:
        print('vazio')

    # faixas_poly.to_file(f'resultados/vetor/faixas-poly-{scm}.gpkg', driver='GPKG')
    # gdf_faixas.plot()

    # break
    

Processando: SCM 3316-214


/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


Processando: SCM 3314-423


/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


Processando: SCM 3314-451


...

/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


Processando: SCM 3316-222


/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


Processando: SCM 3314-441


/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
